Este Notebook muestra la conexión a la base de datos, la ejecución de consultas con pandas, ejemplos de uso de nuestros patrones de diseño y la ejecución de pruebas unitarias.


Celda 1: Verificar Conexión a la Base de Datos

Importa SessionLocal desde src.db.

Ejecuta una consulta corta (SELECT 1) usando session.execute(text("SELECT 1")).

Cierra la sesión y muestra en pantalla si la conexión fue exitosa o reporta el error.

In [1]:
from sqlalchemy import text
from src.db import SessionLocal

try:
    session = SessionLocal()
    result = session.execute(text("SELECT 1")).scalar()
    session.close()
    print("✅ Conexión establecida, SELECT 1 devolvió:", result)
except Exception as e:
    print("❌ Error en la conexión:", e)


✅ Conexión establecida, SELECT 1 devolvió: 1


Celda 2: Ejecución de Consultas SQL con Pandas

Importa la función run_sql desde src.db.

Define una query de ejemplo (p. ej. agrupar ventas por CustomerID).

Llama a run_sql(...) para obtener un DataFrame de pandas con los resultados.

Muestra ese DataFrame en la salida, listo para análisis o visualización.

SE REALIZARON LAS SIGUIETES QUERIES: Consulta simple,
                                     Subconsultas simples,
                                     Subconsulta no correlacionada,
                                     Subconsulta correlacionada,
                                     CTE,
                                     Window function

In [2]:
# Celda 2: Uso de run_sql para obtener DataFrame con pandas
from src.db import run_sql

# Ejemplo: contar ventas por cliente
query = """
SELECT
  CustomerID,
  COUNT(*) AS total_sales
FROM sales
GROUP BY CustomerID
LIMIT 5
"""
df_customers = run_sql(query)
df_customers

,CustomerID,total_sales
0,1,2
1,4,1
2,8,1
3,10,4
4,15,1


In [6]:
#SUBCONSULTA SIMPLE
from src.db import run_sql
query= """
SELECT ProductName, Price 
FROM products
WHERE Price > (SELECT AVG(Price) FROM products)
ORDER BY Price DESC
LIMIT 3; """

df_customers = run_sql(query)
df_customers


,ProductName,Price
0,Shrimp - 31/40,100.0
1,Beef - Inside Round,99.0
2,Bread - Calabrese Baguette,99.0


In [8]:
#SUBCONSULTA NO CORRELACIONADA, NO ES DINAMICO
from src.db import run_sql
query= """SELECT ProductName, Price 
FROM products
WHERE Price > 50.5
ORDER BY Price ASC
LIMIT 5 ;"""

df_customers = run_sql(query)
df_customers

,ProductName,Price
0,Lamb - Ground,51.0
1,Soupfoamcont12oz 112con,51.0
2,Otomegusa Dashi Konbu,51.0
3,Berry Brulee,51.0
4,Wine - Hardys Bankside Shiraz,52.0


In [ ]:
#SUBCONSULTAS CORRELACIONADAS
#MOSTRAR LOS PROUCTOS CUYO PRECIO ES MAYOR AL PROMEDIO DE LOS PRODUCTOS VENDIDOS EN ENERO de 2025
from src.db import run_sql
query= """SELECT DISTINCT
  p.ProductName,
  p.Price,
  s.SalesDate
FROM sales AS s
JOIN products AS p 
  ON s.ProductID = p.ProductID
WHERE 
  s.SalesDate BETWEEN '2025-01-01 00:00:00' 
                  AND '2025-01-31 23:59:59'
  AND p.Price >
    (
      SELECT AVG(p2.Price)
      FROM sales   AS s2
      JOIN products AS p2 
        ON s2.ProductID = p2.ProductID
      WHERE s2.SalesDate BETWEEN '2025-01-01 00:00:00' 
                             AND '2025-01-31 23:59:59'
    );"""

df_customers = run_sql(query)
df_customers

,ProductName,Price,SalesDate
0,Flour - Whole Wheat,74.0,2025-01-21 10:03:23
1,Flour - Whole Wheat,74.0,2025-01-10 05:23:26
2,Flour - Whole Wheat,74.0,2025-01-30 05:48:11
3,Flour - Whole Wheat,74.0,2025-01-08 06:46:12
4,Flour - Whole Wheat,74.0,2025-01-12 19:57:08
...,...,...,...
2390,Soup - Campbells Tomato Ravioli,94.0,2025-01-21 05:02:51
2391,Soup - Campbells Tomato Ravioli,94.0,2025-01-13 18:47:38
2392,Soup - Campbells Tomato Ravioli,94.0,2025-01-05 14:52:11
2393,Soup - Campbells Tomato Ravioli,94.0,2025-01-21 06:07:44


In [14]:
#Producto con el precio maximo por categoria usando una commmon expression table
from src.db import run_sql
query= """WITH max_prices AS (
    SELECT 
      CategoryID,
      MAX(Price) AS PrecioMaximo
    FROM products
    GROUP BY CategoryID
)
SELECT 
  p.ProductName, 
  mp.PrecioMaximo,
  c.CategoryName
FROM max_prices AS mp
JOIN products AS p
  ON p.CategoryID = mp.CategoryID
 AND p.Price      = mp.PrecioMaximo
LEFT JOIN categories AS c 
  ON mp.CategoryID = c.CategoryID
ORDER BY PrecioMaximo DESC;"""

df_customers = run_sql(query)
df_customers 

,ProductName,PrecioMaximo,CategoryName
0,Shrimp - 31/40,100.0,Cereals
1,Bread - Calabrese Baguette,99.0,Dairy
2,Puree - Passion Fruit,99.0,Beverages
3,Beef - Inside Round,99.0,Meat
4,Vanilla Beans,98.0,Poultry
5,Zucchini - Yellow,98.0,Snails
6,Tuna - Salad Premix,97.0,Seafood
7,Bread - Multigrain,97.0,Grain
8,Hot Chocolate - Individual,96.0,Confections
9,Wasabi Powder,94.0,Shell fish


In [16]:
#Con una función de ventana, promedio de ventas por categoria sin perder el detalle por fila
from src.db import run_sql
query= """SELECT s.SalesID, 
	   s.ProductID, 
       p.CategoryID, 
       s.TotalPrice, 
       AVG(s.TotalPrice) 
					OVER (
							PARTITION BY p.CategoryID
													) as PrecioPromedioXCategoria
FROM sales AS s
LEFT JOIN products as p
ON s.ProductID = p.ProductID
LIMIT 5"""

df_customers = run_sql(query)
df_customers 

,SalesID,ProductID,CategoryID,TotalPrice,PrecioPromedioXCategoria
0,6740703,451,1,88.0,158.521353
1,6742402,139,1,24.0,158.521353
2,6743523,105,1,30.0,158.521353
3,6744529,346,1,220.0,158.521353
4,6747844,275,1,325.0,158.521353


Celda 3: Demostración del Patrón Singleton

Importa Database y la instancia global db.

Crea dos instancias (Database() dos veces) y comprueba con assert que ambas son el mismo objeto.

Verifica además que db coincide con Database(), probando que solo existe una conexión compartida.

In [14]:
# Celda 3: Comprobación del patrón Singleton
from src.db import Database, db

inst1 = Database()
inst2 = Database()
assert inst1 is inst2, "❌ Database() debería devolver siempre la misma instancia"
assert inst1 is db, "❌ La instancia global 'db' no coincide con Database()"
print("✅ Patron Singleton verificado: Database() es idéntico a db y comparte engine y SessionLocal")


✅ Patron Singleton verificado: Database() es idéntico a db y comparte engine y SessionLocal


Celda 4: Demostración del Patrón Factory (LectorFactory)

Importa LectorFactory y crea un pequeño DataFrame de prueba.

Guarda ese DataFrame como un CSV temporal (temp_test.csv).

Llama a LectorFactory.get_lector(".csv") para obtener un lector de CSV.

Carga el archivo de prueba con lector.cargar(...) y muestra las primeras filas, validando que el factory entrega el lector adecuado.

In [ ]:
# Celda 4: Ejemplo de Factory Method en LectorFactory
import pandas as pd
from src.ingestion.lector import LectorFactory

# Creación de un DataFrame de prueba y guardado a temp_test.csv
df_temp = pd.DataFrame({
    "SalesID": [1, 2],
    "SalesPersonID": [10, 11],
    "CustomerID": [100, 101],
    "ProductID": [200, 201],
    "Quantity": [5, 3],
    "Discount": [0.1, 0.0],
    "TotalPrice": [450.0, 300.0],
    "SalesDate": ["12:00:00", "13:30:00"],
    "TransactionNumber": ["TXN1", "TXN2"]
})
df_temp.to_csv("temp_test.csv", index=False)

# Seleccionar lector según extensión
lector_csv = LectorFactory.get_lector(".csv")
df_loaded = lector_csv.cargar("temp_test.csv")
print("✅ Factory devolvió LectorCSV y cargó este DataFrame:\n", df_loaded)




✅ Factory devolvió LectorCSV y cargó este DataFrame:
    SalesID  SalesPersonID  CustomerID  ProductID  Quantity  Discount  \
0        1             10         100        200         5       0.1   
1        2             11         101        201         3       0.0   

   TotalPrice SalesDate TransactionNumber  
0       450.0  12:00:00              TXN1  
1       300.0  13:30:00              TXN2  


Celda 5: Demostración del Patrón Builder (SalesBuilder)

Importa Sales.builder() y crea un objeto Sales usando los métodos set_* paso a paso.

Llena cada campo obligatorio (ID, IDs foráneos, cantidad, descuento, total, hora y transacción) y llama a .build().

Muestra la instancia resultante para comprobar que todos los valores se asignaron correctamente.

Intenta un segundo caso donde omite algunos campos obligatorios y capta el ValueError, demostrando la validación interna del builder.

In [16]:
# Celda 5: Ejemplo de patrón Builder para Sales
from src.models.sales import Sales, SalesBuilder
from datetime import time

builder = Sales.builder()
venta = (
    builder
    .set_sales_id(5001)
    .set_sales_person_id(20)
    .set_customer_id(300)
    .set_product_id(400)
    .set_quantity(7.0)
    .set_discount(0.2)
    .set_total_price(560.0)
    .set_sales_date(time(15, 45, 0))
    .set_transaction_number("BUILDERTXN5001")
    .build()
)
print("✅ Builder creó esta instancia de Sales:\n", venta)

# Intentar construir sin campos obligatorios para ver el error
try:
    incomplete = (
        Sales.builder()
        .set_sales_id(5002)
        .set_sales_person_id(21)
        # Falta set_customer_id, etc.
        .build()
    )
except ValueError as err:
    print("🚨 Error esperado al faltar campos:", err)



✅ Builder creó esta instancia de Sales:
 <Sales id=5001 salesDate=15:45:00 total=560.00 transaction=BUILDERTXN5001>
🚨 Error esperado al faltar campos: Faltan campos obligatorios para Sales: ['CustomerID', 'ProductID', 'Quantity', 'Discount', 'TotalPrice', 'SalesDate', 'TransactionNumber']


Celda 6: Ejecución de Pruebas Unitarias con pytest

Llama a !pytest -q --disable-warnings --maxfail=1 desde la propia celda.

Muestra en pantalla el resultado de ejecutar todos los tests definidos en tests/.

Permite verificar “en vivo” que los patrones (Singleton, Builder, etc.) pasan sus pruebas sin fallos

In [17]:
# Celda 6: Ejecutar pytest para validar tests en carpeta tests/
!pytest -q --disable-warnings --maxfail=1


......                                                                   [100%]
6 passed in 1.08s
